<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/aperiodic_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Current fixes to add:

The following are a series of scripts used to generate formatted .txt files for communication with another in-house stroboscope for the SCCS. Of these, #5 is the most powerful, with the capacity to specify multiple steps, the periodicity of the waveform, their durations, start and end effective frequency (in case an aperiodic waveform is chosen), start and end duty cycle, start and end luminance, a square- or sine- wave-modulated signal, and which of 4 LEDs are to be either ON or OFF per each oscillator (4 total). Recent adjustments include proper formatting of TIM to remove DAY stamps, as well as removal of redundant STP durations per oscillator within each row.

1. This version treats LED mapping specific to oscillator, such that the four STPs always go 1 0 0 0, 0 1 0 0, etc; which, after looking at the Session Manager more closely, is likely incorrect, but this version is still being retained for future edits should the LED-specific mapping go awry.

In [ ]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    """Generates Poisson-distributed flash onset times ensuring correct flash count and duty cycle adjustments."""
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    """Calculate effective frequency from generated flash times."""
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    """Finds the optimal initial frequency that produces the desired effective frequency."""
    F = min(target_Fe * 1.2, max_freq)  # Start slightly higher
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Formats a single STP line with all four oscillator parameter blocks concatenated.
    Each block: step duration, wave type, start freq, end freq, start duty, end duty, LED config (4 values), start intensity, end intensity.
    The led_assignments parameter should be a list of four lists, one per oscillator.
    """
    # Build each oscillator's parameter block:
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        # Here we use the same frequency, duty, and brightness for both start and end values.
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    # Combine all four blocks into one STP line (separated by commas, no extra 'STP' tokens in between)
    return f'STP"{",".join(blocks)}"'

def generate_strobe_sequence():
    """Generates a strobe sequence file with proper STP formatting for all four oscillators in one line per step."""
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()
    # We now assume the default LED assignment as per the creator's info:
    # Osc1: 1,0,0,0; Osc2: 0,1,0,0; Osc3: 0,0,1,0; Osc4: 0,0,0,1.
    led_config = [[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]]

    # Header lines
    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    # Determine number of steps (simple approximation)
    num_steps = int(duration * (start_Fe + end_Fe) / 2)
    min_interval = 0.025

    # Generate flash times based on stimulation type
    if stimulation == "Aperiodic":
        F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
    else:
        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps)]
        Fe = start_Fe

    previous_onset = 0
    step_lines = []

    # For each flash time, generate one STP line (which includes the parameters for all 4 oscillators)
    for onset, on_duration in flash_times:
        step_duration = max(onset - previous_onset, min_interval)
        previous_onset = onset + on_duration

        # For simplicity, use start values for the first half and end values for the second half:
        step_freq = start_Fe if onset < duration / 2 else end_Fe
        step_duty = start_d if onset < duration / 2 else end_d
        step_luminance = start_l if onset < duration / 2 else end_l

        stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
        step_lines.append(stp_line)

    output.extend(step_lines)

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

# Run the generator
generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter total sequence duration (seconds): 10
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
Generated strobe_periodic.txt.


2. This version allows for specific LED set mapping per each oscillator, adding an additional 16 questions to the user input prompt. I think this is more correct, compared to the initial fixed single-LED-per-oscillator function above.

In [4]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()

    led_config = get_led_assignments()

    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    num_steps = int(duration * (start_Fe + end_Fe) / 2)
    min_interval = 0.025

    if stimulation == "Aperiodic":
        F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
    else:
        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps)]
        Fe = start_Fe

    previous_onset = 0
    step_lines = []

    for onset, on_duration in flash_times:
        step_duration = max(onset - previous_onset, min_interval)
        previous_onset = onset + on_duration

        step_freq = start_Fe if onset < duration / 2 else end_Fe
        step_duty = start_d if onset < duration / 2 else end_d
        step_luminance = start_l if onset < duration / 2 else end_l

        stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
        step_lines.append(stp_line)

    output.extend(step_lines)

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter total sequence duration (seconds): 10
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_periodic.txt.


3. The following is an adapation of the above [with specific LED/oscillator mapping], while allowing the user to specify distinct steps at which these changes should occur; that is, instead of having to generate multiple transitions using the scripts above and manually appending them, the number of steps and their durations are specified via user input.

In [ ]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM"00:00:{int(total_duration // 60):02}:{total_duration % 60:04.1f}"')
        output.append(f'DUR"{duration:.1f}"')

        num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
        min_interval = 0.025

        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]
        previous_onset = 0

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
            output.append(stp_line)

        total_duration += duration

    file_name = "strobe_sequence.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 3
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Step 2:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 

4. After reviewing some of the parameter restrictions within the Session Manager, it seems that no step can be smaller than 0.2 seconds, which is perhaps why the device was unable to read any of our files. While this will certainly significantly impact our capacity for aperiodicity, the following script is a modification of the above (with precise LED modulation per each oscillator), while making sure that no parameter falls below 0.2 seconds.

In [2]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.200, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)  # Ensure step >= 0.2s
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()

    led_config = get_led_assignments()

    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    num_steps = int(duration * (start_Fe + end_Fe) / 2)

    if stimulation == "Aperiodic":
        F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
    else:
        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps)]
        Fe = start_Fe

    previous_onset = 0
    step_lines = []

    for onset, on_duration in flash_times:
        step_duration = max(onset - previous_onset, 0.200)  # Enforce 0.2s minimum duration
        previous_onset = onset + on_duration

        step_freq = start_Fe if onset < duration / 2 else end_Fe
        step_duty = start_d if onset < duration / 2 else end_d
        step_luminance = start_l if onset < duration / 2 else end_l

        stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
        step_lines.append(stp_line)

    output.extend(step_lines)

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter total sequence duration (seconds): 10
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_periodic.txt.


Well, that's not the case either -- and the explicit code chunk from the creator wasn't readable, either. Stay tuned for more!

5. This should format things so that the STP is not repeated 4x within each row, and is only present at the beginning of each STP row. Further, the DAY TIM is removed.

In [5]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Generates a single STP row where the time value appears only once.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{time:.3f},{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM"00:{int(total_duration // 60):02}:{total_duration % 60:04.1f}"')
        output.append(f'DUR"{duration:.1f}"')

        num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
        min_interval = 0.025

        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]
        previous_onset = 0

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
            output.append(stp_line)

        total_duration += duration

    file_name = "strobe_sequence.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 3
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Step 2:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 

6. This is the same code as above, with 200ms minimum step durations. To change the minimum accepted value, simply change the value within the poisson_flash_times function for min_interval = x. Further, the DAY TIM is removed.

In [6]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.200, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)  # Ensure step >= 0.2s
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Generates a single STP row where the time value appears only once.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{time:.3f},{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM"00:{int(total_duration // 60):02}:{total_duration % 60:04.1f}"')
        output.append(f'DUR"{duration:.1f}"')

        if stimulation == "Aperiodic":
            F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
        else:
            num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
            flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]

        previous_onset = 0
        step_lines = []

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.200)  # Enforce 0.2s minimum duration
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
            step_lines.append(stp_line)

        output.extend(step_lines)
        total_duration += duration

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_periodic.txt.


7. After reviewing a working session from Jimi, it looks like the final change we need to add is how the STP is adjusted incremently. Here is an adjustment of #5, with the very granular timestamps that may not work on the device. An additional adjustment has been made to correctly format the TIM after a maladaption re: TIM.

In [9]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments, stp_index):
    """
    Generates a single STP row where the time value appears only once.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{stp_index},{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{time:.3f},{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0
    stp_index = 1  # Initialize STP index

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM\"00:{int((total_duration + duration) // 60):02}:{(total_duration + duration) % 60:04.1f}\"')
        output.append(f'DUR"{duration:.1f}"')

        num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
        min_interval = 0.025

        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]
        previous_onset = 0

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config, stp_index)
            output.append(stp_line)

            stp_index += 1  # Increment STP index for next row

        total_duration += duration

    file_name = "strobe_sequence.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_sequence.txt.


7. An adjustment of #6, with correct incremental STP adjustments. TIM has also been adjusted per the maladjustment specified above.

In [10]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.200, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)  # Ensure step >= 0.2s
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Generates a single STP row where the time value appears only once.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{time:.3f},{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM\"00:{int((total_duration + duration) // 60):02}:{(total_duration + duration) % 60:04.1f}\"')
        output.append(f'DUR"{duration:.1f}"')

        if stimulation == "Aperiodic":
            F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
        else:
            num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
            flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]

        previous_onset = 0
        step_lines = []

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.200)  # Enforce 0.2s minimum duration
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
            step_lines.append(stp_line)

        output.extend(step_lines)
        total_duration += duration

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_periodic.txt.


8. Further edits to the granular function to ensure that all parameters are in the correct order per Jimi's examples, alongside proper calculation and formatting of TIM and DUR. Bleh!

In [25]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=100):
    """
    Generates a Poisson-distributed sequence of flash times for aperiodic conditions.
    Ensures both start and end frequencies are close to the targets before proceeding.
    """
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):  # Keep regenerating if needed
        times = []
        time = 0

        while time < duration:
            progress = time / duration  # Linear progress from 0 to 1
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)

            # Enforce min/max limits on intervals
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)

            on_duration = interval * (duty_cycle / 100)
            time += interval

            if time < duration:
                times.append((time, on_duration))

        # **Calculate Effective Frequency**
        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)

        # **Check if frequencies are close enough**
        if abs(effective_start_freq - target_start_freq) <= tolerance and abs(effective_end_freq - target_end_freq) <= tolerance:
            return times

    raise ValueError("Failed to generate an aperiodic sequence that meets frequency constraints within the given attempts.")

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty, led_assignments, start_intensity, end_intensity):
    """Formats an STP line using given parameters and ensures correct comma placement between oscillator blocks."""
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{led_str},{start_intensity[osc]},{end_intensity[osc]}"
        blocks.append(block)
    return f'STP"{step_duration:.3f},{",".join(blocks)}"'  # Ensuring correct comma separation between oscillators

def get_led_assignments():
    """Prompts the user for LED assignments per oscillator."""
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config


def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))

        led_config = get_led_assignments()
        step_params.append((duration, periodicity, start_freq, end_freq, start_l, end_l, start_d, end_d, wave_type, led_config))

    # ✅ **TIM and DUR Only Once at the Start**
    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"00:00:{total_duration:.1f}"')
    output.append(f'DUR"{total_duration:.1f}"')

    for step in step_params:
        duration, periodicity, start_freq, end_freq, start_l, end_l, start_d, end_d, wave_type, led_config = step

        if periodicity == "p":
            num_steps_estimated = int(duration * (start_freq + end_freq) / 2)
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq)) for i in range(num_steps_estimated)]
        else:
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        previous_onset = total_elapsed_time

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.025)
            previous_onset += step_duration

            stp_line = format_stp(
                step_duration,
                wave_type,
                [start_freq] * 4, [end_freq] * 4,
                [start_d] * 4, [end_d] * 4,
                led_config,
                [start_l] * 4, [end_l] * 4
            )
            output.append(stp_line)

        total_elapsed_time += duration

    file_name = "attemptgranular.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

# Run the script
generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated attemptgranular.txt.


9. Further edits to the minimum duration function, with correct formatting per Jimi's example.

In [26]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=100):
    """
    Generates a Poisson-distributed sequence of flash times for aperiodic conditions.
    Ensures both start and end frequencies are close to the targets before proceeding.
    """
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):  # Keep regenerating if needed
        times = []
        time = 0

        while time < duration:
            progress = time / duration  # Linear progress from 0 to 1
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)

            # Enforce min/max limits on intervals
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)

            on_duration = interval * (duty_cycle / 100)
            time += interval

            if time < duration:
                times.append((time, on_duration))

        # **Calculate Effective Frequency**
        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)

        # **Check if frequencies are close enough**
        if abs(effective_start_freq - target_start_freq) <= tolerance and abs(effective_end_freq - target_end_freq) <= tolerance:
            return times

    raise ValueError("Failed to generate an aperiodic sequence that meets frequency constraints within the given attempts.")

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty, led_assignments, start_intensity, end_intensity):
    """Formats an STP line using given parameters and ensures correct comma placement between oscillator blocks."""
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{led_str},{start_intensity[osc]},{end_intensity[osc]}"
        blocks.append(block)
    return f'STP"{max(step_duration, 0.200):.3f},{",".join(blocks)}"'  # Ensuring step duration is at least 200ms

def get_led_assignments():
    """Prompts the user for LED assignments per oscillator."""
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))

        led_config = get_led_assignments()
        step_params.append((duration, periodicity, start_freq, end_freq, start_l, end_l, start_d, end_d, wave_type, led_config))

    # **TIM and DUR Only Once at the Start**
    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"00:00:{total_duration:.1f}"')
    output.append(f'DUR"{total_duration:.1f}"')

    for step in step_params:
        duration, periodicity, start_freq, end_freq, start_l, end_l, start_d, end_d, wave_type, led_config = step

        if periodicity == "p":
            num_steps_estimated = int(duration * (start_freq + end_freq) / 2)
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq)) for i in range(num_steps_estimated)]
        else:
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        previous_onset = total_elapsed_time

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.200)  # Ensuring minimum step duration of 200ms
            previous_onset += step_duration

            stp_line = format_stp(
                step_duration,
                wave_type,
                [start_freq] * 4, [end_freq] * 4,
                [start_d] * 4, [end_d] * 4,
                led_config,
                [start_l] * 4, [end_l] * 4
            )
            output.append(stp_line)

        total_elapsed_time += duration

    file_name = "attempt200.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

# Run the script
generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated attempt200.txt.


It seems that, after reviewing output from the DESKTOP SESSION MANAGER, that when the LEDs for one oscillator are selected, it turns off that LED in all other oscillators. This might be linked to our issue with the device not being able to read our generated codes, albeit the multiple formatting corrections.

In [27]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=100):
    """
    Generates a Poisson-distributed sequence of flash times for aperiodic conditions.
    Ensures both start and end frequencies are close to the targets before proceeding.
    """
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):  # Keep regenerating if needed
        times = []
        time = 0
        while time < duration:
            progress = time / duration  # Linear progress from 0 to 1
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)
            # Enforce min/max limits on intervals
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)
            on_duration = interval * (duty_cycle / 100)
            time += interval
            if time < duration:
                times.append((time, on_duration))
        # Calculate effective frequencies for first and second halves
        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)
        # Check if frequencies are close enough
        if abs(effective_start_freq - target_start_freq) <= tolerance and abs(effective_end_freq - target_end_freq) <= tolerance:
            return times
    raise ValueError("Failed to generate an aperiodic sequence that meets frequency constraints within the given attempts.")

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty, led_assignments, start_intensity, end_intensity):
    """Formats an STP line using given parameters and ensures correct comma placement between oscillator blocks.
       The step duration is forced to be at least 0.200 seconds."""
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{led_str},{start_intensity[osc]},{end_intensity[osc]}"
        blocks.append(block)
    return f'STP"{max(step_duration, 0.200):.3f},{",".join(blocks)}"'

def get_led_assignments():
    """Prompts the user for LED assignments per oscillator."""
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def reconcile_led_assignments(led_config):
    """
    For each LED index (each column in the LED assignment matrices),
    if multiple oscillators have that LED turned on, only the first occurrence remains on;
    the others are forced to 0.
    """
    num_leds = len(led_config[0])
    for led in range(num_leds):
        found = False
        for osc in range(len(led_config)):
            if led_config[osc][led] == 1:
                if not found:
                    found = True
                else:
                    led_config[osc][led] = 0
    return led_config

def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))
        led_config = get_led_assignments()
        # Dynamically reconcile LED assignments so that if a given LED is on in one oscillator,
        # it is forced off in the others.
        led_config = reconcile_led_assignments(led_config)
        step_params.append((duration, periodicity, start_freq, end_freq, start_l, end_l, start_d, end_d, wave_type, led_config))

    # TIM and DUR appear only once at the top (for the entire sequence)
    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"00:00:{total_duration:.1f}"')
    output.append(f'DUR"{total_duration:.1f}"')

    for step in step_params:
        duration, periodicity, start_freq, end_freq, start_l, end_l, start_d, end_d, wave_type, led_config = step

        if periodicity == "p":
            num_steps_estimated = int(duration * (start_freq + end_freq) / 2)
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq)) for i in range(num_steps_estimated)]
        else:
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        previous_onset = total_elapsed_time
        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.200)  # Enforce minimum step duration of 200ms
            previous_onset += step_duration
            stp_line = format_stp(
                step_duration,
                wave_type,
                [start_freq] * 4, [end_freq] * 4,
                [start_d] * 4, [end_d] * 4,
                led_config,
                [start_l] * 4, [end_l] * 4
            )
            output.append(stp_line)
        total_elapsed_time += duration

    file_name = "z_single_oscillator.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))
    print(f"Generated {file_name}.")

# Run the script
generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Generated z_single_oscillator.txt.


Rats, still unreadable -- but hopefully we're getting closer with the explicit examples put forth by the desktop session manager.

In [28]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=100):
    """
    Generates a Poisson-distributed sequence of flash times for aperiodic conditions.
    Ensures both start and end frequencies are close to the targets before proceeding.
    """
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):
        times = []
        time = 0
        while time < duration:
            progress = time / duration  # Linear progress from 0 to 1
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)
            on_duration = interval * (duty_cycle / 100)
            time += interval
            if time < duration:
                times.append((time, on_duration))
        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)
        if abs(effective_start_freq - target_start_freq) <= tolerance and abs(effective_end_freq - target_end_freq) <= tolerance:
            return times
    raise ValueError("Failed to generate an aperiodic sequence that meets frequency constraints within the given attempts.")

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty, led_assignments, start_intensity, end_intensity):
    """
    Formats an STP line using the given parameters.
    Ensures each oscillator block is comma-separated and that step_duration is at least 0.200 seconds.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{led_str},{start_intensity[osc]},{end_intensity[osc]}"
        blocks.append(block)
    return f'STP"{max(step_duration, 0.200):.3f},{",".join(blocks)}"'

def get_led_assignments():
    """
    Prompts the user for LED assignments for each of the four oscillators.
    """
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def reconcile_led_assignments(led_config):
    """
    For each LED (each column), if more than one oscillator has that LED on,
    only the first oscillator (in order) keeps it on; all others are set to off.
    """
    num_leds = len(led_config[0])
    for led in range(num_leds):
        found = False
        for osc in range(len(led_config)):
            if led_config[osc][led] == 1:
                if not found:
                    found = True
                else:
                    led_config[osc][led] = 0
    return led_config

def generate_strobe_sequence():
    """
    Main function to generate the strobe sequence.
    Prompts for multiple step parameters and outputs one TIM and one DUR for the entire sequence,
    followed by a series of STP lines.
    """
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))
        led_config = get_led_assignments()
        # Dynamically reconcile LED assignments across oscillators
        led_config = reconcile_led_assignments(led_config)
        step_params.append((duration, periodicity, start_freq, end_freq, start_l, end_l, start_d, end_d, wave_type, led_config))

    # Compute total sequence duration and output TIM and DUR once at the top
    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"00:00:{total_duration:.1f}"')
    output.append(f'DUR"{total_duration:.1f}"')

    # Process each step
    for step in step_params:
        duration, periodicity, start_freq, end_freq, start_l, end_l, start_d, end_d, wave_type, led_config = step

        if periodicity == "p":
            num_steps_estimated = int(duration * (start_freq + end_freq) / 2)
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq)) for i in range(num_steps_estimated)]
        else:
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        previous_onset = total_elapsed_time
        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.200)  # Enforce minimum step duration of 200ms
            previous_onset += step_duration
            stp_line = format_stp(
                step_duration,
                wave_type,
                [start_freq] * 4, [end_freq] * 4,
                [start_d] * 4, [end_d] * 4,
                led_config,
                [start_l] * 4, [end_l] * 4
            )
            output.append(stp_line)
        total_elapsed_time += duration

    file_name = "maybebaby.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))
    print(f"Generated {file_name}.")

# Run the script
generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Generated maybebaby.txt.


In [29]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle,
                          min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=100):
    """
    Generates a Poisson-distributed sequence of flash times for aperiodic conditions.
    It computes a current cycle length that linearly interpolates between the start
    and end cycle lengths and then draws intervals from an exponential distribution.
    If the effective frequencies (calculated from the flashes in the first and second halves)
    are within tolerance of the targets, the candidate sequence is returned.
    """
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):
        times = []
        time = 0.0
        while time < duration:
            progress = time / duration  # Linear progress from 0 to 1
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)
            # Enforce min and max limits on interval
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)
            on_duration = interval * (duty_cycle / 100)
            time += interval
            if time < duration:
                times.append((time, on_duration))
        # Calculate effective frequencies in the first and second halves:
        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)
        if (abs(effective_start_freq - target_start_freq) <= tolerance and
            abs(effective_end_freq - target_end_freq) <= tolerance):
            return times
    raise ValueError("Failed to generate an aperiodic sequence that meets frequency constraints within the given attempts.")

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty,
               led_assignments, start_intensity, end_intensity):
    """
    Formats a single STP line. The step duration is forced to be at least 200ms.
    The four oscillator blocks are concatenated with commas between them.
    """
    # Format each oscillator’s block
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{led_str},{start_intensity[osc]},{end_intensity[osc]}"
        blocks.append(block)
    # Ensure a minimum step duration of 0.100 seconds
    return f'STP"{max(step_duration, 0.100):.3f},{",".join(blocks)}"'

def get_led_assignments():
    """
    Prompts the user for LED assignments for each oscillator.
    Returns a list of four lists (one per oscillator), each containing four integers.
    """
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def adjust_led_assignments(led_config):
    """
    Enforces that only one oscillator is 'active' by keeping only one oscillator’s LED values
    and setting all LED values of the other oscillators to 0.

    The active oscillator is chosen as the first oscillator that has any LED set to 1.
    If none are active, oscillator 1 is chosen by default.
    """
    active_index = 0
    for i in range(4):
        if sum(led_config[i]) > 0:
            active_index = i
            break
    # For every oscillator other than the active one, set LED values to zeros.
    for i in range(4):
        if i != active_index:
            led_config[i] = [0, 0, 0, 0]
    return led_config

def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0.0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))

        # Get LED assignments for all 4 oscillators
        led_config = get_led_assignments()
        # Enforce that only one oscillator remains active (others are turned off)
        led_config = adjust_led_assignments(led_config)

        # Append all step parameters as a tuple.
        step_params.append((duration, periodicity, start_freq, end_freq,
                            start_l, end_l, start_d, end_d, wave_type, led_config))

    # Compute overall total duration and output TIM and DUR (only once)
    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"00:00:{total_duration:.1f}"')
    output.append(f'DUR"{total_duration:.1f}"')

    # Process each step and generate STP lines
    for step in step_params:
        (duration, periodicity, start_freq, end_freq,
         start_l, end_l, start_d, end_d, wave_type, led_config) = step

        if periodicity == "p":
            # For periodic conditions, estimate flash times using a fixed frequency
            num_steps_estimated = int(duration * (start_freq + end_freq) / 2)
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq))
                           for i in range(num_steps_estimated)]
        else:
            # For aperiodic conditions, use the Poisson-based generator.
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        # Set the starting point for the step (cumulative elapsed time)
        previous_onset = total_elapsed_time

        # For each flash, compute the step duration (ensuring a minimum of 100ms)
        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.100)
            previous_onset += step_duration

            stp_line = format_stp(
                step_duration,
                wave_type,
                [start_freq] * 4, [end_freq] * 4,
                [start_d] * 4, [end_d] * 4,
                led_config,
                [start_l] * 4, [end_l] * 4
            )
            output.append(stp_line)
        total_elapsed_time += duration

    file_name = "z_test.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))
    print(f"Generated {file_name}.")

# Run the script
generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Generated z_test.txt.


The issue may be that inactive oscillators, while being set to 0, still have some parameters kept active. I have added this fix below, to try and match the output as closely as possible to the example from Jimi:

TIM"00:00:15.0"
DUR"15.0"
STP"5.0,1,10.00,20.00,50,90,1,1,0,0,0,100,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"
STP"5.0,1,20.00,5.00,90,30,1,1,1,1,100,30,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"
STP"5.0,2,5.00,2.00,30,30,0,0,1,1,30,50,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"
                                                                  

In [38]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle,
                          min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=1000):
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):
        times = []
        time = 0.0
        while time < duration:
            progress = time / duration
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)
            on_duration = interval * (duty_cycle / 100)
            time += interval
            if time < duration:
                times.append((time, on_duration))

        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)

        if (abs(effective_start_freq - target_start_freq) <= tolerance and
            abs(effective_end_freq - target_end_freq) <= tolerance):
            return times
    raise ValueError("Failed to generate an aperiodic sequence within constraints.")

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty,
               led_assignments, start_intensity, end_intensity):
    """
    Formats a single STP line, ensuring inactive oscillators retain frequency & duty cycle
    but have wave type, LEDs, and intensity set to 0.
    """
    blocks = []
    active_osc = any(any(leds) for leds in led_assignments)

    for osc in range(4):
        if any(led_assignments[osc]):
            block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{','.join(map(str, led_assignments[osc]))},{start_intensity[osc]},{end_intensity[osc]}"
        else:
            block = f"0,{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},0,0,0,0,0,0"
        blocks.append(block)

    return f'STP"{max(step_duration, 0.1):.1f},{",".join(blocks)}"'

def get_led_assignments():
    """
    Prompts the user for LED assignments for each oscillator.
    """
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def adjust_led_assignments(led_config):
    """
    Ensures only one oscillator is active.
    """
    active_index = 0
    for i in range(4):
        if sum(led_config[i]) > 0:
            active_index = i
            break

    for i in range(4):
        if i != active_index:
            led_config[i] = [0, 0, 0, 0]
    return led_config

def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0.0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))

        led_config = get_led_assignments()
        led_config = adjust_led_assignments(led_config)

        step_params.append((duration, periodicity, start_freq, end_freq,
                            start_l, end_l, start_d, end_d, wave_type, led_config))

    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"00:00:{total_duration:.1f}"')
    output.append(f'DUR"{total_duration:.1f}"')

    for step in step_params:
        (duration, periodicity, start_freq, end_freq,
         start_l, end_l, start_d, end_d, wave_type, led_config) = step

        if periodicity == "p":
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq))
                           for i in range(int(duration * start_freq))]
        else:
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        previous_onset = total_elapsed_time

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.100)
            previous_onset += step_duration

            stp_line = format_stp(step_duration, wave_type, [start_freq]*4, [end_freq]*4, [start_d]*4, [end_d]*4, led_config, [start_l]*4, [end_l]*4)
            output.append(stp_line)
        total_elapsed_time += duration

    with open("z_test_finale.txt", "w") as f:
        f.write("\n".join(output))
    print("Generated z_test_finale.txt.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 2
Step 1:
  Enter step duration (seconds): .1
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Step 2:
  Enter step duration (seconds): .1
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 1

I think we've narrowed down all possible permutations of incorrect decimal placing, STP formatting, and virtually everything else, after significant comparisons to the output provided by the session manager ...

In [40]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle,
                        min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=1000):
    """Generates an aperiodic Poisson-distributed sequence of flashes."""
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):
        times = []
        time = 0.0
        while time < duration:
            progress = time / duration
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)
            on_duration = interval * (duty_cycle / 100)
            time += interval
            if time < duration:
                times.append((time, on_duration))

        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)

        if (abs(effective_start_freq - target_start_freq) <= tolerance and
            abs(effective_end_freq - target_end_freq) <= tolerance):
            return times
    raise ValueError("Failed to generate an aperiodic sequence within constraints.")

def format_time(total_seconds):
    """Formats TIM as hh:mm:ss.s with leading zeros."""
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    fraction = total_seconds - int(total_seconds)
    return f"{minutes:02}:{seconds:02}:{fraction:.1f}"

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty,
               led_assignments, start_intensity, end_intensity):
    """Formats a single STP line."""
    blocks = []
    for osc in range(4):
        if any(led_assignments[osc]):
            block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{','.join(map(str, led_assignments[osc]))},{start_intensity[osc]},{end_intensity[osc]}"
        else:
            block = f"0,{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},0,0,0,0,0,0"
        blocks.append(block)
    return f'STP"{max(step_duration, 0.1):.1f},{",".join(blocks)}"'

def get_led_assignments():
    """Prompts the user for LED assignments per oscillator."""
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def adjust_led_assignments(led_config):
    """Ensures only one oscillator is active."""
    active_index = 0
    for i in range(4):
        if sum(led_config[i]) > 0:
            active_index = i
            break
    for i in range(4):
        if i != active_index:
            led_config[i] = [0, 0, 0, 0]
    return led_config

def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0.0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))

        led_config = get_led_assignments()
        led_config = adjust_led_assignments(led_config)

        step_params.append((duration, periodicity, start_freq, end_freq,
                            start_l, end_l, start_d, end_d, wave_type, led_config))

    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"{format_time(total_duration)}"')
    output.append(f'DUR"{total_duration:.1f}"')

    for step in step_params:
        (duration, periodicity, start_freq, end_freq,
         start_l, end_l, start_d, end_d, wave_type, led_config) = step

        if periodicity == "p":
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq))
                           for i in range(int(duration * start_freq))]
        else:
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        previous_onset = total_elapsed_time

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.100)
            previous_onset += step_duration

            stp_line = format_stp(step_duration, wave_type, [start_freq]*4, [end_freq]*4,
                                  [start_d]*4, [end_d]*4, led_config, [start_l]*4, [end_l]*4)
            output.append(stp_line)
        total_elapsed_time += duration

    with open("z_test_omni.txt", "w") as f:
        f.write("\n".join(output))
    print("Generated z_test_omni.txt.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 2
Step 1:
  Enter step duration (seconds): .1
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Step 2:
  Enter step duration (seconds): .1
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 1

In [81]:
import os
import difflib

# Define file paths
file1_path = "/10_22.txt"  # Replace with actual file names if needed
file2_path = "/content/z_test_nospace.txt"

# Check if both files exist
if not os.path.exists(file1_path) or not os.path.exists(file2_path):
    print("❌ One or both files are missing.")
else:
    # Read file contents
    with open(file1_path, "r", encoding="utf-8") as f1, open(file2_path, "r", encoding="utf-8") as f2:
        lines1 = f1.readlines()
        lines2 = f2.readlines()

    # Compare line-by-line
    print("\n🔍 Differences Found:\n")
    diff = difflib.unified_diff(lines1, lines2, fromfile="File 1", tofile="File 2", lineterm="")

    has_diff = False
    for line in diff:
        has_diff = True
        print(line.strip())

    if not has_diff:
        print("✅ No visible differences detected in lines.")

    # Perform character-by-character comparison for subtle differences
    print("\n🔬 Checking for hidden differences (spaces, encoding, special characters):")
    for i in range(min(len(lines1), len(lines2))):
        if lines1[i] != lines2[i]:
            print(f"\n⚠️ Difference at line {i+1}:")
            print(f"File 1: {repr(lines1[i])}")
            print(f"File 2: {repr(lines2[i])}")

    if len(lines1) != len(lines2):
        print("\n⚠️ Files have a different number of lines!")


🔍 Differences Found:

--- File 1
+++ File 2
@@ -1,5 +1,4 @@
TIM"00:00:00.2"
DUR"0.2"
STP"0.1,1,10.00,10.00,50,50,1,1,1,1,50,50,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"
-STP"0.1,1,10.00,10.00,50,50,1,1,1,1,50,50,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"
-
+STP"0.1,1,10.00,10.00,50,50,1,1,1,1,50,50,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"

🔬 Checking for hidden differences (spaces, encoding, special characters):

⚠️ Difference at line 4:
File 1: 'STP"0.1,1,10.00,10.00,50,50,1,1,1,1,50,50,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"\n'
File 2: 'STP"0.1,1,10.00,10.00,50,50,1,1,1,1,50,50,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"'

⚠️ Files have a different number of lines!


Apparently, the final difference between otherwise identical script was an invisible formatting issue regarding line spacing. Thank goodness for Python's ability to compare files byte-for-byte!

In [80]:
import numpy as np

def poisson_flash_times(target_start_freq, target_end_freq, duration, duty_cycle,
                        min_interval=0.025, max_interval_factor=3.0, tolerance=0.05, max_attempts=1000):
    """Generates an aperiodic Poisson-distributed sequence of flashes."""
    start_cycle_length = 1.0 / target_start_freq
    end_cycle_length = 1.0 / target_end_freq

    for _ in range(max_attempts):
        times = []
        time = 0.0
        while time < duration:
            progress = time / duration
            current_cycle_length = start_cycle_length + progress * (end_cycle_length - start_cycle_length)
            interval = np.random.exponential(current_cycle_length * 0.8)
            interval = min(interval, current_cycle_length * max_interval_factor)
            interval = max(interval, min_interval)
            on_duration = interval * (duty_cycle / 100)
            time += interval
            if time < duration:
                times.append((time, on_duration))

        effective_start_freq = len([t for t in times if t[0] < duration * 0.5]) / (duration * 0.5)
        effective_end_freq = len([t for t in times if t[0] >= duration * 0.5]) / (duration * 0.5)

        if (abs(effective_start_freq - target_start_freq) <= tolerance and
            abs(effective_end_freq - target_end_freq) <= tolerance):
            return times
    raise ValueError("Failed to generate an aperiodic sequence within constraints.")

def format_time(total_seconds):
    """Formats TIM as hh:mm:ss.s with leading zeros."""
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    fraction = total_seconds - int(total_seconds)
    return f"00:{minutes:02}:{seconds:02}.{int(fraction * 10)}"

def format_stp(step_duration, wave_type, start_freq, end_freq, start_duty, end_duty,
               led_assignments, start_intensity, end_intensity):
    """Formats a single STP line."""
    blocks = []
    for osc in range(4):
        if any(led_assignments[osc]):
            block = f"{wave_type},{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},{','.join(map(str, led_assignments[osc]))},{start_intensity[osc]},{end_intensity[osc]}"
        else:
            block = f"0,{start_freq[osc]:.2f},{end_freq[osc]:.2f},{start_duty[osc]},{end_duty[osc]},0,0,0,0,0,0"
        blocks.append(block)
    return f'STP"{max(step_duration, 0.1):.1f},{",".join(blocks)}"'

def get_led_assignments():
    """Prompts the user for LED assignments per oscillator."""
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def adjust_led_assignments(led_config):
    """Ensures only one oscillator is active."""
    active_index = 0
    for i in range(4):
        if sum(led_config[i]) > 0:
            active_index = i
            break
    for i in range(4):
        if i != active_index:
            led_config[i] = [0, 0, 0, 0]
    return led_config

def normalize_output(output_lines):
    """Ensures strict formatting consistency with session manager output."""
    return [line.rstrip() + "\n" for line in output_lines]

def generate_strobe_sequence():
    """Main function to generate the strobe sequence."""
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_elapsed_time = 0.0
    step_params = []

    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        periodicity = input("  Enter periodicity (p=Periodic, a=Aperiodic): ").strip().lower()
        start_freq = float(input("  Enter start effective frequency (Hz): "))
        end_freq = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = int(input("  Enter wave type (0=Off, 1=Square, 2=Sine): "))

        led_config = get_led_assignments()
        led_config = adjust_led_assignments(led_config)

        step_params.append((duration, periodicity, start_freq, end_freq,
                            start_l, end_l, start_d, end_d, wave_type, led_config))

    total_duration = sum([step[0] for step in step_params])
    output.append(f'TIM"{format_time(total_duration).strip()}"')
    output.append(f'DUR"{total_duration:.1f}"'.strip())  # Ensure no trailing spaces


    for step in step_params:
        (duration, periodicity, start_freq, end_freq,
         start_l, end_l, start_d, end_d, wave_type, led_config) = step

        if periodicity == "p":
            flash_times = [(i * (1.0 / start_freq), (start_d / 100) * (1.0 / start_freq))
                           for i in range(int(duration * start_freq))]
        else:
            flash_times = poisson_flash_times(start_freq, end_freq, duration, start_d)

        previous_onset = total_elapsed_time

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.100)
            previous_onset += step_duration

            stp_line = format_stp(step_duration, wave_type, [start_freq]*4, [end_freq]*4,
                                  [start_d]*4, [end_d]*4, led_config, [start_l]*4, [end_l]*4)
            output.append(stp_line)
        total_elapsed_time += duration

    normalized_output = normalize_output(output)

    with open("z_test_nospace.txt", "w", encoding="utf-8", newline="") as f:
        f.write("\n".join(output))

    print("Generated z_test_nospace.txt with strict session manager formatting.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 2
Step 1:
  Enter step duration (seconds): .1
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (0=Off, 1=Square, 2=Sine): 1
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 0
  LED 2: 0
  LED 3: 0
  LED 4: 0
Step 2:
  Enter step duration (seconds): .1
  Enter periodicity (p=Periodic, a=Aperiodic): p
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 1

Our comparison script insists the script generated is identical to that of the session manager. Let's see ...

Outside of crazy possibilities like a missing newline or trailing space, there aren't really any differences between files. Let's check encrypton- and byte- level stuff then to figure out what the issue might be, since the files take up different amounts of space albeit having the same amount of characters (pretty much).

In [79]:
import chardet

# Define file path
file_path = "/10_22.txt"

# Step 1: Detect Encoding
with open(file_path, "rb") as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)

encoding = result["encoding"]
confidence = result["confidence"]

print(f"🔍 Detected Encoding: {encoding} (Confidence: {confidence:.2f})")

# Step 2: Check Bit Depth (8-bit vs. 16-bit)
try:
    decoded_text = raw_data.decode(encoding, errors="ignore")
    byte_length = len(raw_data)
    character_length = len(decoded_text)

    if byte_length / character_length == 2:
        bit_depth = "16-bit (UTF-16 or similar)"
    elif byte_length / character_length == 1:
        bit_depth = "8-bit (UTF-8, ANSI, etc.)"
    else:
        bit_depth = "Unknown"

    print(f"🔎 Bit Depth: {bit_depth}")

except Exception as e:
    print(f"❌ Error decoding file: {e}")
    bit_depth = "Unknown"

# Step 3: Check for Hidden Characters
if raw_data.startswith(b'\xef\xbb\xbf'):
    print("⚠️ BOM (Byte Order Mark) detected at the beginning of the file.")
if raw_data.endswith(b'\n'):
    print("✔️ File has a trailing newline (matches standard text files).")
else:
    print("⚠️ File is missing a trailing newline!")

# Step 4: Display first few lines for inspection
try:
    print("\n📜 First few lines of the file:")
    with open(file_path, "r", encoding=encoding) as f:
        for i, line in enumerate(f.readlines()):
            print(f"{i+1:02}: {repr(line)}")
            if i >= 4:  # Show only the first 5 lines
                break
except Exception as e:
    print(f"❌ Error reading file: {e}")

print("\n✅ Analysis Complete! Now, compare this with your generated file.")


🔍 Detected Encoding: ascii (Confidence: 1.00)
🔎 Bit Depth: 8-bit (UTF-8, ANSI, etc.)
⚠️ File is missing a trailing newline!

📜 First few lines of the file:
01: 'TIM"00:00:00.2"\n'
02: 'DUR"0.2"\n'
03: 'STP"0.1,1,10.00,10.00,50,50,1,1,1,1,50,50,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"\n'
04: 'STP"0.1,1,10.00,10.00,50,50,1,1,1,1,50,50,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0,0,10.00,10.00,50,50,0,0,0,0,0,0"\n'
05: '                                                                                                                                                                                                             '

✅ Analysis Complete! Now, compare this with your generated file.


Time to check hexcode, etc?